# Import Libraries

In [115]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [116]:
# Reading the data
data = pd.read_excel('../data/responses_data_500.xlsx')

In [117]:
# Checking the head of the data
data.head()

description    answer_category_num  \
0  ูุฒูุฑ ุงูุฎุงุฑุฌูุฉ ุงููุจูุงูู ุฌุจุฑุงู ุจุงุณูู ูุงู ูู ุณูุณู...  Religious affiliation   
1  ูุฒูุฑ ุงูุฎุงุฑุฌูุฉ ุงููุจูุงูู ุฌุจุฑุงู ุจุงุณูู ูุงู ูู ุณูุณู...  Religious affiliation   
2  ูุฒูุฑ ุงูุฎุงุฑุฌูุฉ ุงููุจูุงูู ุฌุจุฑุงู ุจุงุณูู ูุงู ูู ุณูุณู...  Religious affiliation   
3  ูุฒูุฑ ุงูุฎุงุฑุฌูุฉ ุงููุจูุงูู ุฌุจุฑุงู ุจุงุณูู ูุงู ูู ุณูุณู...                 Racist   
4  ูุฒูุฑ ุงูุฎุงุฑุฌูุฉ ุงููุจูุงูู ุฌุจุฑุงู ุจุงุณูู ูุงู ูู ุณูุณู...  Religious affiliation   

   question_id_id  
0               1  
1               1  
2               1  
3               1  
4               1

In [118]:
# The shape of the data
data.shape

(500, 3)

In [119]:
data.description = data.description.apply(str.strip)

In [120]:
# The columns of the data
data.columns

Index(['description', 'answer_category_num', 'question_id_id'], dtype='object')

In [121]:
data.description.duplicated().sum()

126

In [122]:
clean_data = data.copy()
clean_data  = clean_data[['description', 'answer_category_num']]
clean_data.head(2)

description    answer_category_num
0  ูุฒูุฑ ุงูุฎุงุฑุฌูุฉ ุงููุจูุงูู ุฌุจุฑุงู ุจุงุณูู ูุงู ูู ุณูุณู...  Religious affiliation
1  ูุฒูุฑ ุงูุฎุงุฑุฌูุฉ ุงููุจูุงูู ุฌุจุฑุงู ุจุงุณูู ูุงู ูู ุณูุณู...  Religious affiliation

In [123]:
clean_data.description.value_counts()

ูุฒูุฑ ุงูุฎุงุฑุฌูุฉ ุงููุจูุงูู ุฌุจุฑุงู ุจุงุณูู ูุงู ูู ุณูุณูุฉ ุชุบุฑูุฏุงุชู ุนูุจ ุงุฎุชุชุงู ุงูููุฉ ุงูุนุฑุจูุฉ ุงูุชููููุฉ ุงูุงูุชุตุงุฏูุฉ ูุงูุงุฌุชูุงุนูุฉ ููุณูุง ุชุฌโฆ             7
ุณูุฑูุฉ ุจูุฏ ุงูุญุถุงุฑุงุช ุชุฑุจุทูุง ุจุนููุฉ ุงู ุจุญููุงู                                                                                               6
ูุง ููุจ ุงูุช ููุจ ูุฌูุณู ูุง ุชุญูู ุนู ุงูุฅูุณุงููู                                                                                               5
ุงุฎู ุงูุญุงุฌ ุงุฐุง ุดุนุฑุช ุงูู ูุญุฑุฌุง ูู ุงูุงูุชูุงุฏุงุช ูุชุตุฑูุญ ุฌุจุฑุงู ุจุงุณูู ูุง ุฏุงุนู ูุฃู ุชูุงุฌู ุงูููุชูุฏูู                                               4
ููุง ูููุง ุชููู ุงูุจุฑุบูุซู ุงุฎููุฌู ุณุงูู ุงุญุฐุฑูู ูุทู ุงุดูุงุกูุง ุงููุตุฑููู ููุงูู ูุงุงุงุงุงุงุน ุฏู ุดุงุนุฑ ููููุฌู ุทูุจ ููุนู ุจูู ุนูู

In [124]:
sum(clean_data.description.value_counts() > 1)

94

In [125]:
# removing duplicated descrptions/ comments
clean_data.drop_duplicates(subset='description', inplace=True)
clean_data.reset_index(drop=True, inplace=True)

In [126]:
clean_data

description    answer_category_num
0    ูุฒูุฑ ุงูุฎุงุฑุฌูุฉ ุงููุจูุงูู ุฌุจุฑุงู ุจุงุณูู ูุงู ูู ุณูุณู...  Religious affiliation
1            ุณูุฑูุฉ ุจูุฏ ุงูุญุถุงุฑุงุช ุชุฑุจุทูุง ุจุนููุฉ ุงู ุจุญููุงู                 Racist
2    ุงุฎู ุงูุญุงุฌ ุงุฐุง ุดุนุฑุช ุงูู ูุญุฑุฌุง ูู ุงูุงูุชูุงุฏุงุช ูุชุต...                 Normal
3    ูุง ููู ุชุนูุด ุจูุง ูุง ุชูุจ ูุชู ููู ููุงุฑ ูุจูุฑุฉ ูููู...                 Normal
4    ูุฐุง ุงูุจุทู ุงูุฐู ูุงุชู ูุฌุงุฐู ุจุญูุงุชู ูุชุญูุง ุงูุช ูุง ...  Religious affiliation
..                                                 ...                    ...
369  ููุนูู ูู ูุงุจูู ูุงุจู ุนููุชู ูุงุจู ุงุณุฑุงุฆูู ุนูู ุงุจู...                Mockery
370                                       ุนุงูุงุชู ูุงูุทู                Mockery
371  ุฃูุฑ ุนูููุงุช ุงูููู ุฌุจุฑุงู ุจุงุณูู ูููุฐ ุงูุงูุฑ ุงูููุงู...                 Normal
372          ุญุท ุตุฑูุงูุฉ ุจุชูู ูุชู ููู ุฎููู ูููุช ููุงู ููุจ                Violent
373  ุงูููุฑุจุง ููุงุณุฉ ุตูุตุงู  ุงูุฎุงุฑุฌูุฉ ุณูุงุญุฉ ูุณูุฑ ูุฌุจุฑุง...                Mockery

[374 rows x 2 columns]

In [127]:
# checking how many comments have different votes
for i, row in clean_data.iterrows():

    comment = row.description

    # getting the dataframe for that comment
    temp_df = data[data.description == comment]
    
    # how many unique answers that comment has
    n_ = temp_df.answer_category_num.nunique()

    # changing the answer_category_num to the most frequent one
    if n_ >= 2:
        most_voted_label = temp_df.answer_category_num.value_counts().index[0]
        clean_data.at[i, 'answer_category_num'] = most_voted_label

In [128]:
clean_data

description    answer_category_num
0    ูุฒูุฑ ุงูุฎุงุฑุฌูุฉ ุงููุจูุงูู ุฌุจุฑุงู ุจุงุณูู ูุงู ูู ุณูุณู...  Religious affiliation
1            ุณูุฑูุฉ ุจูุฏ ุงูุญุถุงุฑุงุช ุชุฑุจุทูุง ุจุนููุฉ ุงู ุจุญููุงู                Violent
2    ุงุฎู ุงูุญุงุฌ ุงุฐุง ุดุนุฑุช ุงูู ูุญุฑุฌุง ูู ุงูุงูุชูุงุฏุงุช ูุชุต...                 Normal
3    ูุง ููู ุชุนูุด ุจูุง ูุง ุชูุจ ูุชู ููู ููุงุฑ ูุจูุฑุฉ ูููู...                 Normal
4    ูุฐุง ุงูุจุทู ุงูุฐู ูุงุชู ูุฌุงุฐู ุจุญูุงุชู ูุชุญูุง ุงูุช ูุง ...  Religious affiliation
..                                                 ...                    ...
369  ููุนูู ูู ูุงุจูู ูุงุจู ุนููุชู ูุงุจู ุงุณุฑุงุฆูู ุนูู ุงุจู...                Mockery
370                                       ุนุงูุงุชู ูุงูุทู                Mockery
371  ุฃูุฑ ุนูููุงุช ุงูููู ุฌุจุฑุงู ุจุงุณูู ูููุฐ ุงูุงูุฑ ุงูููุงู...                 Normal
372          ุญุท ุตุฑูุงูุฉ ุจุชูู ูุชู ููู ุฎููู ูููุช ููุงู ููุจ                Violent
373  ุงูููุฑุจุง ููุงุณุฉ ุตูุตุงู  ุงูุฎุงุฑุฌูุฉ ุณูุงุญุฉ ูุณูุฑ ูุฌุจุฑุง...                Mockery

[374 rows x 2 columns]

In [129]:
# Generated Class
clean_data.answer_category_num.value_counts()

Mockery                  112
Normal                    78
Violent                   68
Sexual harrasment         41
Religious affiliation     38
Racist                    37
Name: answer_category_num, dtype: int64

In [130]:
# Checking the info
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   description          374 non-null    object
 1   answer_category_num  374 non-null    object
dtypes: object(2)
memory usage: 6.0+ KB


In [131]:
# Removing stop words for each tweet
clean_data['tweet_no_stopwords'] = 'x'
stop_words = set(stopwords.words('arabic')) 
for count, tweet in enumerate(clean_data.description):
    word_tokens = word_tokenize(tweet)
    filtered_tweet = []
    for word in word_tokens:
        if word not in stop_words:
            filtered_tweet.append(word)
    joined_filtered_tweet = " ".join(filtered_tweet)
    clean_data.tweet_no_stopwords[count] = joined_filtered_tweet

In [132]:
clean_data.head(2)

description    answer_category_num  \
0  ูุฒูุฑ ุงูุฎุงุฑุฌูุฉ ุงููุจูุงูู ุฌุจุฑุงู ุจุงุณูู ูุงู ูู ุณูุณู...  Religious affiliation   
1          ุณูุฑูุฉ ุจูุฏ ุงูุญุถุงุฑุงุช ุชุฑุจุทูุง ุจุนููุฉ ุงู ุจุญููุงู                Violent   

                                  tweet_no_stopwords  
0  ูุฒูุฑ ุงูุฎุงุฑุฌูุฉ ุงููุจูุงูู ุฌุจุฑุงู ุจุงุณูู ูุงู ุณูุณูุฉ ุช...  
1          ุณูุฑูุฉ ุจูุฏ ุงูุญุถุงุฑุงุช ุชุฑุจุทูุง ุจุนููุฉ ุงู ุจุญููุงู

# Training the Model

In [223]:
# y is Class which is dependent on X Tweet
X = clean_data['tweet_no_stopwords']
y = clean_data['answer_category_num']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [224]:
y_train.shape, y_test.shape

((317,), (57,))

In [225]:
y_train.value_counts()

Mockery                  88
Normal                   71
Violent                  54
Sexual harrasment        37
Religious affiliation    36
Racist                   31
Name: answer_category_num, dtype: int64

In [226]:
y_test.value_counts()

Mockery                  24
Violent                  14
Normal                    7
Racist                    6
Sexual harrasment         4
Religious affiliation     2
Name: answer_category_num, dtype: int64

In [227]:
def show_results(y_true, y_pred):
    print("Confusion Matrix")
    print(metrics.confusion_matrix(y_true, y_pred))

    print("Classification Report")
    print(metrics.classification_report(y_true, y_pred, )) 

### SVC

In [228]:
# Using pipiles for machine learning flow

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)

In [229]:
show_results(y_test, predictions)

Confusion Matrix
[[8 9 0 1 4 2]
 [3 3 0 0 0 1]
 [1 2 1 0 0 2]
 [2 0 0 0 0 0]
 [2 2 0 0 0 0]
 [4 4 0 0 1 5]]
Classification Report
                       precision    recall  f1-score   support

              Mockery       0.40      0.33      0.36        24
               Normal       0.15      0.43      0.22         7
               Racist       1.00      0.17      0.29         6
Religious affiliation       0.00      0.00      0.00         2
    Sexual harrasment       0.00      0.00      0.00         4
              Violent       0.50      0.36      0.42        14

             accuracy                           0.30        57
            macro avg       0.34      0.21      0.21        57
         weighted avg       0.41      0.30      0.31        57



### Naive Bayes

In [230]:
# Using pipiles for machine learning flow
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)

In [231]:
show_results(y_test, predictions)

Confusion Matrix
[[14  9  0  0  0  1]
 [ 5  2  0  0  0  0]
 [ 4  2  0  0  0  0]
 [ 2  0  0  0  0  0]
 [ 3  1  0  0  0  0]
 [10  3  0  0  0  1]]
Classification Report
                       precision    recall  f1-score   support

              Mockery       0.37      0.58      0.45        24
               Normal       0.12      0.29      0.17         7
               Racist       0.00      0.00      0.00         6
Religious affiliation       0.00      0.00      0.00         2
    Sexual harrasment       0.00      0.00      0.00         4
              Violent       0.50      0.07      0.12        14

             accuracy                           0.30        57
            macro avg       0.16      0.16      0.12        57
         weighted avg       0.29      0.30      0.24        57



In [160]:
tst =  "ุงูุช  ูุงุญุฏ ุญููุฑ ููุง ุจุชุณุชุญู ุง"

In [161]:
text_clf.predict([tst])[0]

'Mockery'

In [61]:
wrong = 0
mock_viol = 0
correct_classified = 0

mock_viol_list = ['Violent', 'Mockery']

for count, test_tweet in enumerate(X_test):
    
    y_test_answer = y_test.values[count]
    y_test_prediction = predictions[count]

    # getting the correct classified comments
    if y_test_answer == y_test_prediction:
        correct_classified += 1

    if y_test_answer != y_test_prediction:
        print("Tweet: {}".format(test_tweet))
        print("Real Prediction: {} ||  Model Prediction: {}\n".format(y_test_answer, y_test_prediction))

        if y_test_answer in mock_viol_list and y_test_prediction in mock_viol_list:
            mock_viol += 1
        wrong += 1

print("Correct Classified {}".format(correct_classified))        
print("Misclassified {}".format(wrong))
mock_viol

Tweet: ูุช ุจุบูุถู
Real Prediction: Mockery ||  Model Prediction: Normal

Tweet: ุขุฎ ุฌุจุฑุงู ุจุงุณูู ุดู ูุงูุฑ ุนุงูู ุงูุช ูุนูู ููุง ูุฌุฑ
Real Prediction: Religious affiliation ||  Model Prediction: Violent

Tweet: ุงููู ูููููุง
Real Prediction: Religious affiliation ||  Model Prediction: Normal

Tweet: ุฑุฏูุฏ ุงูุดุนุจ ุงููุจูุงูู ููุงูุฉ ุฑุฆูุณ ุงูุญุฒุจ ุชูุญูุฏ ุงูุนุฑุจ ูุงู ุงูุช ูุนุฑูู ูุฐูุจู ูุฑู ููุฑู ุถุฏ ููู
Real Prediction: Normal ||  Model Prediction: Mockery

Tweet: ุงูู ุจุงุณูู ุงุฑุชูุจ ุฌุฑููุฉ ุงู ุญูู ุดู ุจุฑุงุช ุงูุณูุงู
Real Prediction: Violent ||  Model Prediction: Mockery

Tweet: ุณูุจูู ูุนูู ูุชุญููู ููุทู ุงูุนุฏุงูุฉ ูุฌููุน ุฃุจูุงุก ุงูุทุงุฆูุฉ ุงููุนุฑูููุฉ
Real Prediction: Racist ||  Model Prediction: Normal

Tweet: ุฌูุด ุงุจู ุดุญุงุทุฉ ุงููุตูุฑู ุงููุฐุฑ ุงูุฌูุด ุงููุญูุฏ ุจุงูุนุงูู ููู ุจููุชู ุดุนุจู ูุจูู ุง

83

In [71]:
X_test.iloc[0]

1236    ุดู ุชูุจุฑููุ ููุจุฑูููู ุงูุฏูุนุฉ ุงูุฅูุงุฑุงุชูุฉุ ุจูู ุชุณุช...
1062                                             ูุช ุจุบูุถู
2530         ุขุฎ ุฌุจุฑุงู ุจุงุณูู ุดู ูุงูุฑ ุนุงูู ุงูุช ูุนูู ููุง ูุฌุฑ
619                                            ุงูุช ุตุฑูุงูู
773                                           ุงููู ูููููุง
                              ...                        
299                            ุฌุจุฑุงู ุจุงุณูู ุณููู ุฌุงุจ ููุฑุจุง
2000                                     ููุงูู ุตุญูุญ ููุงุณู
2432                ููุฉ ุงูููุงุญุฉ ุนูุฏูุง ุชุญุงุตุฑ ุงูุนุงูุฑุฉ ุงูุนูุฉ
1539                     ุญูุง ุงููู ูุจูุงู ูุดุนุจ ูุจูุงู ุงูุนุธูู
705     ูุนูู ุจูู ูุฒูุฑ ุงููุตุงูุญุฉ ุนูู ุญูุฏุฑ ุจุงุน ููุงุฏูุง ุงู ...
Name: tweet_no_stopwords, Length: 618, dtype: object

# Getting questions id's for correctly classified questions by model

In [86]:
list_of_correct_ids = []

for i, test_tweet in enumerate(X_test):
    y_test_answer = y_test.values[i]
    y_test_prediction = predictions[i]

    # getting the correct classified comments
    if y_test_answer == y_test_prediction:
        # getting the original tweet before doing any preprocessing ex BOW removal
        description = clean_data[clean_data['tweet_no_stopwords'] == X_test.iloc[i]]['description'].values[0]

        # getting the question id
        question_id_id = data[data['description'] == description]['question_id_id'].values[0]

        list_of_correct_ids.append(question_id_id)

# Saving the Model

In [27]:
import pickle
# save the model to disk
filename = 'finalized_model_SVC.sav'
pickle.dump(text_clf, open(filename, 'wb'))

In [74]:
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8393162393162393
